# Darstellung des Temperaturverlaufes von 24h an einer Wetterstation

Von Lea Pellikan   |   Matrikel-Nr.: 3062311   |   Abgabe: 15.11.2020

Im folgendem Jupyter Notebook soll ein Barchart erstellt werden, das den stündlichen Temperaturverlauf eines ausgewählten Tages an einer ausgewählten Wetterstation darstellt. Die dafür verwendeten Daten stammen von der Quelle *************** . Diese Quelle wird von ******+ gepflegt und stellt, unter anderem, die stündlich gemessenen Temperaturen von allen Wetterstationen in Deutschland da.  Die Daten wurden aus Performanzgründen nicht direkt von Kafka importiert, sondern per CSV-Datei in eine MongoDB gespeichert. Aus dieser werden die benötigten Daten ausgelesen.

## Import der Libraries

Im folgendem wird die Library **pymongo** für die Verbindung zur MongoDB verwendet und **pandas** für das Verarbeiten der Collection. Außerdem wird **plotly.express** zur Darstellung der Daten verwendet. 

In [ ]:
import pandas as pd
import plotly.express as px
from pymongo import MongoClient

## Verbindungsaufbau zur MongoDB

Im Folgendem wird die Verbindung zur MongoDB aufgebaut, wobei die Library **pymongo** verwendet wird. In dieser Anwnedung läuft diese in Docker.

In [ ]:
client = MongoClient('mongodb://localhost:27017/')
stations = client.bda.stations
weather = client.bda.weather

Die methode **get_collection** erstellt ein Dictonary aus der Collection die ausgelesen wurde.

In [ ]:
def get_collection(collection):
    documents = []
    cursor = collection.find({})
    for document in cursor:
        documents.append(document)
    return documents

Im folgendem wird die Methode **get_collection** verwendet um mit Hilfe von pandas das Dataframe *stations_df* mit den Informationen der Wetterstationen und das Dataframe *weather_df* mit den Wetterinformationen gefüllt wird. Außerdem wird die Spalte *STATION_ID* zu *station_id* umbenannt, um eine einheitliche Benennung zu erhalten.

In [ ]:
stations_df = pd.DataFrame(get_collection(stations))
stations_df = stations_df.rename(columns={"STATION_ID": "station_id"})

In [ ]:
weather_df = pd.DataFrame(get_collection(weather))

## Erläuterung der Daten

Danach erfolgt die Zusammenführung der oben erwähnten Dataframes, wobei die station_id das gemeinsamme Attribut ist. Wie in der Ausgabe zu sehen ist, enthält das Dataframe jetzt folgende Daten:
* die IDs der Wetterstationen (station_id), 
* das Datum inklusive Uhrzeit, zu dem die folgende Temperatur gemessen wurde (date), 
* die Zeitspanne des Datensatzes (FROM_DATE, TO_DATE), 
* das in diesem Beispiel vernachlässigt werden kann, 
* die Höhe der Messstation (STATION_HEIGHT), 
* die Koordinaten der Wetterstation (LAT,LON), 
* den Namen der Station (STATION_NAME) und 
* das Bundesland, in der die Wetterstation sich befindet (STATE).

In [ ]:
weather_stations = pd.merge(weather_df,stations_df, on="station_id", how="left")
weather_stations

In der Methode *temp_by_date()* wird nach die Temperaturen des übergebenden Datums an der Station, dessen ID übergeben wurde, gefiltert.

In [ ]:
def temp_by_date(df,date,stationid):
    result= df.loc[df['date'].dt.normalize() == date ]
    result2 =  result.loc[result['station_id']== stationid ]
    return result2

Hier kommt die Methode *temp_by_date* zur Verwendung. Als Beispielparameter wird hier die ID der Wetterstation in Darmstadt (917) und das Datum 20.08.2020 übergeben.
In der Ausgabe ist zu sehen, dass nurnoch die Informationen dieser Wetterstation an diesem Datum abgebildet werden.

In [ ]:
tempbydate = temp_by_date(weather_stations,"2020-08-20 00:00:00",917)
tempbydate

## Konfiguration des Diagrams

In der Methode *calculate_plot* werden die Daten des oben abgebildete Dataframes in ein Bar-Chart transformiert. Hierbei stellt die X-Achse die Uhrzeit da und die Y-Achse die Temperatur der jeweiligen Stunde. In der Überschrift wird sowohl das Datum als auch der Name der Wetterstation angezeigt. Die Farbe der Balken ist an die Höhe der Temperatur angepasst.

In [ ]:
def calculate_plot(tempbydate):
    fig2 =  px.bar(tempbydate,
             y= "temperature", 
             x= "date",
               title = 'Temperaturverlauf am ' + str(tempbydate["date"].dt.date.iloc[0]) +
                ' in ' + str(tempbydate["STATION_NAME"].iloc[0] ) ,
                hover_data=['station_id', 'STATION_NAME'], color='temperature')
    fig2.show()

In [ ]:
calculate_plot(tempbydate)